## **Feature:** [FEATURE_NAME]

**Names:** [YOUR_NAME]

### **What it does**
[Brief description]

### **Helper Functions**
[List Helper Functions]

In [6]:
import numpy as np
import pandas as pd


In [7]:
def zscore_outliers_mask(s: pd.Series, z_thresh: float = 3.0) -> pd.Series:
    """True where |z| > z_thresh; safe for zero/NaN std."""
    if not pd.api.types.is_numeric_dtype(s):
        return pd.Series(False, index=s.index)
    x = s.astype(float)
    mu = x.mean(skipna=True)
    sd = x.std(skipna=True)
    if not np.isfinite(sd) or sd == 0:
        return pd.Series(False, index=s.index)
    z = (x - mu) / sd
    return z.abs() > z_thresh

def iqr_bounds(s: pd.Series, k: float = 1.5):
    x = s.astype(float)
    q1 = x.quantile(0.25)
    q3 = x.quantile(0.75)
    iqr = q3 - q1
    return q1 - k * iqr, q3 + k * iqr

def iqr_outliers_mask(s: pd.Series, k: float = 1.5) -> pd.Series:
    if not pd.api.types.is_numeric_dtype(s):
        return pd.Series(False, index=s.index)
    low, high = iqr_bounds(s, k)
    x = s.astype(float)
    return (x < low) | (x > high)


In [ ]:
def zscore_report(df: pd.DataFrame, z_thresh=3.0):
    numeric_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
    rows = len(df)
    records = []
    for c in numeric_cols:
        m = zscore_outliers_mask(df[c], z_thresh)
        records.append({
            "column": c,
            "z_thresh": z_thresh,
            "outliers": int(m.sum()),
            "rows": rows,
            "pct_outliers": round(100*m.mean(), 3)
        })
    print(pd.DataFrame(records))
    return df

def iqr_report(df: pd.DataFrame, iqr_k=1.5):
    numeric_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
    rows = len(df)
    records = []
    for c in numeric_cols:
        m = iqr_outliers_mask(df[c], iqr_k)
        low, high = iqr_bounds(df[c], iqr_k)
        records.append({
            "column": c,
            "iqr_k": iqr_k,
            "lower_bound": low,
            "upper_bound": high,
            "outliers": int(m.sum()),
            "rows": rows,
            "pct_outliers": round(100*m.mean(), 3)
        })
    print(pd.DataFrame(records))
    return df


In [30]:
def zscore_outliers_rows(df: pd.DataFrame, z_thresh=3.0) -> pd.DataFrame:
    """
    Return all rows where ANY numeric column is an outlier by Z-score.
    """
    numeric_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
    if not numeric_cols:
        return pd.DataFrame()  # no numeric columns
    
    mask_union = pd.Series(False, index=df.index)
    for c in numeric_cols:
        mask_union |= zscore_outliers_mask(df[c], z_thresh)
    
    print(df.loc[mask_union])
    return df

def iqr_outliers_rows(df: pd.DataFrame, iqr_k=1.5) -> pd.DataFrame:
    """
    Return all rows where ANY numeric column is an outlier by IQR.
    """
    numeric_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
    if not numeric_cols:
        return pd.DataFrame()
    
    mask_union = pd.Series(False, index=df.index)
    for c in numeric_cols:
        mask_union |= iqr_outliers_mask(df[c], iqr_k)
    
    print(df.loc[mask_union])
    return df

In [ ]:
def outlier_percent_tables(
    df: pd.DataFrame,
    z_thresh: float = 3.0,
    iqr_k: float = 1.5,
):
    num_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
    rows = len(df)

    # Z-score table
    z_records = []
    for c in num_cols:
        m = zscore_outliers_mask(df[c], z_thresh)
        z_records.append({
            "column": c,
            "z_thresh": z_thresh,
            "outliers": int(m.sum()),
            "rows": rows,
            "pct_outliers": round(100 * m.mean(), 3),
        })
    z_table = pd.DataFrame(z_records).sort_values("pct_outliers", ascending=False)

    # IQR table
    iqr_records = []
    for c in num_cols:
        m = iqr_outliers_mask(df[c], iqr_k)
        low, high = iqr_bounds(df[c], iqr_k)
        iqr_records.append({
            "column": c,
            "iqr_k": iqr_k,
            "lower_bound": float(low),
            "upper_bound": float(high),
            "outliers": int(m.sum()),
            "rows": rows,
            "pct_outliers": round(100 * m.mean(), 3),
        })
    iqr_table = pd.DataFrame(iqr_records).sort_values("pct_outliers", ascending=False)

    return z_table.reset_index(drop=True), iqr_table.reset_index(drop=True)


In [16]:
def outlier_union_table(
    df: pd.DataFrame,
    z_thresh: float = 3.0,
    iqr_k: float = 1.5,
):
    num_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
    rows = len(df)
    records = []
    for c in num_cols:
        m_z = zscore_outliers_mask(df[c], z_thresh)
        m_iqr = iqr_outliers_mask(df[c], iqr_k)
        m_any = m_z | m_iqr
        records.append({
            "column": c,
            "z_thresh": z_thresh,
            "iqr_k": iqr_k,
            "outliers_any": int(m_any.sum()),
            "rows": rows,
            "pct_outliers_any": round(100 * m_any.mean(), 3),
        })
    return pd.DataFrame(records).sort_values("pct_outliers_any", ascending=False).reset_index(drop=True)


In [26]:
def recommend_treatments(
    df: pd.DataFrame,
    union_tbl: pd.DataFrame,
    skew_abs_threshold: float = 0.75
):
    """
    Returns a table with: column, pct_outliers_any, skew, unique_ratio, recommendation, rationale
    """
    rows = len(df)
    recs = []
    for _, r in union_tbl.iterrows():
        c = r["column"]
        pct = r["pct_outliers_any"]
        s = df[c].astype(float)
        skew = float(s.skew(skipna=True)) if pd.api.types.is_numeric_dtype(s) else np.nan
        unique_ratio = df[c].nunique(dropna=True) / rows if rows else 0.0

        # Default
        recommendation = "none"
        rationale = "No action."

        if pct < 1.0:
            recommendation = "drop_rows"
            rationale = "<1% outliers — removing affected rows is low risk."
        elif pct < 10.0:
            recommendation = "clip"
            rationale = "1–10% outliers — capping to IQR bounds preserves most data."
        elif pct < 30.0:
            if skew > skew_abs_threshold:
                recommendation = "log"
                rationale = f"{pct:.1f}% outliers with right-skew (skew={skew:.2f}) — log transform often helps."
            elif skew < -skew_abs_threshold:
                recommendation = "standardize"
                rationale = f"{pct:.1f}% outliers with left-skew (skew={skew:.2f}) — scaling recommended."
            else:
                recommendation = "standardize"
                rationale = f"{pct:.1f}% outliers, moderate skew (skew={skew:.2f}) — standardize."
        else:
            recommendation = "investigate"
            rationale = f">{pct:.1f}% outliers — check data quality/units before treating."

        # If the column looks categorical-like (very few unique values), warn:
        if unique_ratio < 0.05 and pd.api.types.is_numeric_dtype(df[c]):
            rationale += " Column has low unique ratio; may be categorical encoded."

        recs.append({
            "column": c,
            "pct_outliers_any": pct,
            "skew": round(skew, 3) if np.isfinite(skew) else np.nan,
            "unique_ratio": round(float(unique_ratio), 4),
            "recommendation": recommendation,
            "rationale": rationale,
        })

    return pd.DataFrame(recs).sort_values("pct_outliers_any", ascending=False).reset_index(drop=True)


In [ ]:
def treat_outliers(df: pd.DataFrame,
    cols=None,
    policy: str = "none",     # "drop_rows" | "clip" | "log" | "standardize" | "none"
    z_thresh: float = 3.0,
    iqr_k: float = 1.5
):
    """
    Returns a NEW DataFrame with chosen outlier treatment applied.
    """
    if cols is None:
        cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
    out = df.copy()

    # --- Build union mask across methods ---
    union_mask = pd.Series(False, index=df.index)
    for c in cols:
        if not pd.api.types.is_numeric_dtype(df[c]):
            continue
        m_z = zscore_outliers_mask(df[c], z_thresh)
        m_iqr = iqr_outliers_mask(df[c], iqr_k)
        union_mask |= (m_z | m_iqr)

    if policy == "drop_rows":
        return out.loc[~union_mask].copy()

    if policy == "clip":
        for c in cols:
            if not pd.api.types.is_numeric_dtype(out[c]): 
                continue
            low, high = iqr_bounds(out[c], iqr_k)
            out[c] = out[c].clip(lower=low, upper=high)
        return out

    if policy == "log":
        for c in cols:
            if not pd.api.types.is_numeric_dtype(out[c]): 
                continue
            s = out[c].astype(float)
            minv = s.min(skipna=True)
            shift = -(minv) + 1e-6 if minv < 0 else 0.0
            out[c] = np.log1p(s + shift)
        return out

    if policy == "standardize":
        for c in cols:
            if not pd.api.types.is_numeric_dtype(out[c]): 
                continue
            s = out[c].astype(float)
            mu, sd = s.mean(skipna=True), s.std(skipna=True)
            out[c] = (s - mu) / sd if sd and np.isfinite(sd) else s*0.0
        return out

    return out


In [24]:
import matplotlib.pyplot as plt

def plot_box(df: pd.DataFrame, cols=None):
    """
    Draw a boxplot for each numeric column in cols.
    """
    if cols is None:
        cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
    for c in cols:
        if not pd.api.types.is_numeric_dtype(df[c]): 
            continue
        plt.figure()
        plt.boxplot(df[c].dropna().values, vert=True)
        plt.title(f"Boxplot — {c}")
        plt.xlabel(c)
        plt.ylabel("value")
        plt.show()


In [ ]:
# df = pd.read_csv("cleaned_smoke.csv")  # your cleaned data

# Example: drop rows with outliers
df_drop = treat_outliers(df, policy="drop_rows")
print("Original shape:", df.shape, "-> after drop:", df_drop.shape)

# Example: clip values
df_clip = treat_outliers(df, policy="clip")

# Example: log transform
df_log = treat_outliers(df, policy="log")

# Example: standardize
df_std = treat_outliers(df, policy="standardize")

# Example: plot
plot_box(df)